In [1]:
import pandas as pd
import os 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch import  nn
from torchvision import transforms ,datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler

from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
InputDim=784
HiddenDim=200
OutputDim=20


InputDimCNN=1
HiddenDimCNN=3
OutputDimCNN=1

LR_rate=1e-3
Num_epochs=10
batch_size=512


In [3]:
# TrainData=datasets.CIFAR10(root="/home/yash/CNN_Data",train=True,transform=transforms.ToTensor(),download=True)
# TestData=datasets.CIFAR10(root="/home/yash/CNN_Data",train=False,transform=transforms.ToTensor(),download=True) 
#Use this for the CIFAR10 Dataset

TrainData=datasets.MNIST(root="/home/yash/CNN_Data",train=True,transform=transforms.ToTensor(),download=True)
TestData=datasets.MNIST(root="/home/yash/CNN_Data",train=False,transform=transforms.ToTensor(),download=True) 
TrainDataSet=DataLoader(TrainData,batch_size=batch_size)
TestDataSet=DataLoader(TestData,batch_size=batch_size)

In [4]:
class VARAutoencodersLinear(nn.Module):
    def __init__(self,input_dim,hidden_dim,z_dim):
        super().__init__()

        #Flatten
        self.Flat=nn.Flatten()

        #Encoder
        self.img_2hid=nn.Linear(input_dim,hidden_dim)
        self.hid_2mu=nn.Linear(hidden_dim,z_dim)
        self.hid_2sigma=nn.Linear(hidden_dim,z_dim)

        #Decoder

        self.z_2hid=nn.Linear(z_dim,hidden_dim)
        self.hid_2img=nn.Linear(hidden_dim,input_dim)


        self.Relu=nn.ReLU()
        self.sigmoid=nn.Sigmoid()
        pass
    
    def Encoder(self,x):
        hd=self.Relu(self.img_2hid(x))
        mu,sigma=self.hid_2mu(hd),self.hid_2sigma(hd)
        return(mu,sigma)

    def Decoder(self,z):

        h=self.Relu(self.z_2hid(z))
        out=self.sigmoid(self.hid_2img(h))
        return(out)

    def forward(self,Dataset):

        mu,sigma=self.Encoder(Dataset)
        epsilon=torch.rand_like(sigma)
        z_parameterized=mu+sigma*epsilon

        x_recontructed=self.Decoder(z_parameterized)

        return(x_recontructed,mu,sigma)

In [6]:
vaeLin=VARAutoencodersLinear(InputDim,HiddenDim,OutputDim).to(device)
Optimizer=Adam(vaeLin.parameters(),lr=LR_rate)
loss_fn=nn.BCELoss(reduction="sum")

In [7]:
for epochs in range(Num_epochs):
    loop=tqdm(enumerate(TrainDataSet),colour="Blue")

    for i, (x,_) in loop:
        x=x.to(device).view(x.shape[0],InputDim)
        x_recontructed,mu,sigma=vaeLin(x)


        recontructed_loss=loss_fn(x_recontructed,x)
        kl_div=-torch.sum(1+torch.log(sigma.pow(2))-mu.pow(2)-sigma.pow(2))

        loss=recontructed_loss+kl_div

        Optimizer.zero_grad()
        loss.backward()
        Optimizer.step()
        
        loop.set_postfix(loss=loss.item(),epoch=epochs+1)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]